In [1]:
from tqdm.notebook import tqdm
from Portable import osp

import json

In [2]:
ref_base = r"../../sln/VQA_SA/parsed_rect.json"
entities = r"./entities"
output = r"./merged_entities.json"

In [3]:
with open(ref_base, "r", encoding="utf8") as reader_ref:
    data:dict = json.load(reader_ref)

In [4]:
merge_list = list()
for k, v in data.items():
    json_path = osp.j(entities, osp.refmt(k, ".json"))
    with open(json_path, "r", encoding="utf8") as reader:
        rec_data = json.load(reader)
    for i, ques in enumerate(v):
        question = ques["question"]
        recs = rec_data[i]
        merge_list.append({
            "image_name": k,
            "question": question,
            "objects": recs
        })

筛除 LLM 提取异常 如 “拍摄者” “图中场景” “你” “我” “他”

In [5]:
excepts = ["图中的场景", "图中场景", "该图片的", "图片中场景", "图片中的场景", "当前图片", "图片场景"]
excepts2 = [
    "你", "我", "他", "它", "她", "图片内容", "图片中的其他区域", "图片左侧", "图片右侧", "图片中央", 
    "拍摄这张图片的人", "拍摄这张照片的人", "拍摄人所在的位置", "拍摄位置", "当前拍摄点", "拍照人",
    "拍照的人", "拍照者", "正在拍照的人", "拍摄时间", "城市", "背景", "所有人", "打的"    
]
excepts3 = [r"拍摄者$", r"拍摄视角$", r"拍摄人$", r"拍摄的人$"]
counter = 0; c_counter = 0

import re
rest_list = list()
for i, e in enumerate(tqdm(merge_list)):
    objects = e["objects"]
    rms = list()
    for k in list(objects.keys()):
        cond = any([e in k for e in excepts])
        cond = cond or any([e == k for e in excepts2])
        cond = cond or any([len(re.findall(e, k)) > 0 for e in excepts3])
        c_counter += 1
        if not cond: rest_list.append(k); continue 
        counter += 1
        rms.append(k)
    for k in rms: del objects[k]
    merge_list[i]["objects"] = objects
print(c_counter, "-", counter, "=", c_counter - counter)
# with open("./sample/rest_entities.json", "w", encoding="utf8") as writer:
#     json.dump(rest_list, writer, indent=4, ensure_ascii=False)

  0%|          | 0/6549 [00:00<?, ?it/s]

12412 - 789 = 11623


坐标化 筛除没有坐标的

In [6]:
coord_pattern = r"\[(\d+)[^\d]+(\d+)[^\d]+(\d+)[^\d]+(\d+)\]"

counter = 0; c_counter = 0
for i, e in enumerate(tqdm(merge_list)):
    objects = e["objects"]
    rms = list()
    for k in list(objects.keys()):
        ans = re.findall(coord_pattern, objects[k])
        if len(ans) == 0: rms.append(k); counter += 1
        else: objects[k] = [int(e) for e in ans[0]]; c_counter += 1  
    for rm in rms: del objects[rm]
    merge_list[i]["objects"] = objects

print(c_counter + counter, "-", counter, "=", c_counter)

  0%|          | 0/6549 [00:00<?, ?it/s]

11623 - 62 = 11561


不能在问题中匹配的 用 spacy 处理匹配至相似度最高的等长 token 串

In [7]:
import spacy

nlp = spacy.load("zh_core_web_md")

nf_counter = 0; counter = 0
for i, e in enumerate(tqdm(merge_list)):
    rp_dict = dict()
    for k in e["objects"]:
        counter += 1
        if k in e["question"]: continue
        nf_counter += 1
        search = nlp(k)
        sentence = nlp(e["question"])
        highest = -99; record = ""
        for j in range(len(sentence) - len(search)):
            window = nlp("".join([e.text for e in sentence[j:j+len(search)]]))
            similarity = window.similarity(search)
            if similarity > highest: highest = similarity; record = window.text
        rp_dict[k] = record
    for k, v in rp_dict.items():
        merge_list[i]["objects"][v] = merge_list[i]["objects"][k]
        del merge_list[i]["objects"][k]
print(nf_counter, "/", counter)

  0%|          | 0/6549 [00:00<?, ?it/s]

/tmp/ipykernel_3222415/3405980412.py:17: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity = window.similarity(search)


1145 / 11561


In [8]:
from Portable import console as cs
cs.pakprint(merge_list[0])

pak: {
    "image_name": "12_0_101_0000001_240107.jpg",
    "question": "穿白色衣服的人在穿棕色衣服的人的哪个方向？请从前方、后方、左方、右方、上方、下方、左前方、右前方、左后方、右后方中选择",
    "objects": {
        "穿白色衣服的人": [
            443,
            926,
            485,
            1037
        ],
        "穿棕色衣服的人": [
            534,
            926,
            578,
            1018
        ]
    }
}


In [9]:
with open(output, "w", encoding="utf8") as writer:
    json.dump(merge_list, writer, indent=4, ensure_ascii=False)